In [1]:
INPUT_NODES = 48
OUTPUT_NODES = 2

# Reference: https://discuss.pynq.io/t/tutorial-pynq-dma-part-2-using-the-dma-from-pynq/3134

In [2]:
from pynq import Overlay, allocate, PL, ps, get_rails, DataRecorder
import os
import numpy as np
# from fxpmath import Fxp
import time

PL.reset()

cwd = os.getcwd()
bitstream_path = os.path.join(cwd, "mlp.bit")

# ol.free()
ol = Overlay(bitstream_path)

ol.ip_dict.keys()


dict_keys(['dma', 'mlp_0', 'ps'])

In [3]:
print(ps.Clocks.fclk0_mhz) # Using PL0 clock from Block Diagram
print(ps.Clocks.fclk1_mhz)
print(ps.Clocks.fclk2_mhz)
print(ps.Clocks.fclk3_mhz)
# ps.Clocks.fclk0_mhz = 1
# ps.Clocks.fclk1_mhz = 1
# ps.Clocks.fclk2_mhz = 1
# ps.Clocks.fclk3_mhz = 1

rails = get_rails()

recorder = DataRecorder(rails["PSINT_FP"].power)

149.9985
99.999
99.999
99.999


In [4]:
dma = ol.dma
dma_send = dma.sendchannel
dma_receive = dma.recvchannel

dma.sendchannel.stop()
dma.recvchannel.stop()
dma.sendchannel.start()
dma.recvchannel.start()

input_buffer = allocate(shape=(INPUT_NODES,), dtype=np.float32)
output_buffer = allocate(shape=(OUTPUT_NODES,), dtype=np.float32)

mlp = ol.mlp_0

CONTROL_REGISTER = 0x0
mlp.write(CONTROL_REGISTER, 0x81)

# dma.register_map
# mlp.register_map

In [5]:
import time

def get_fpga_output(input_data):
    try:
        if len(input_data) != INPUT_NODES:
            print("Input data length doesn't match neural network input")
            return
        
        for i in range(len(input_data)):
            input_buffer[i] = input_data[i]
        # send input to FPGA and wait for output
        
#         print(f"Output buffer: {output_buffer}")
#         print(f"Sending data: {input_buffer}")
        dma_send.transfer(input_buffer)
        dma_send.wait()
        dma_receive.transfer(output_buffer)
        dma_receive.wait()
        print(f"Output buffer: {output_buffer}")
        # --------------- TIMING ---------------- #
        action = np.argmax(output_buffer)
        
    except Exception as e:
        print("FPGA error")
        print(e)
        return
    return action
    

In [6]:
test_input = np.array([-0.479, -1.2179, -0.4232, -1.8137, -1.6443, -2.4633, -0.25, -0.6336, -0.0521, 1.7223, 1.2101, 1.5145, -0.37152, -0.9549599999999998, -0.27877, 0.01579999999999997, -0.06824000000000001, -0.12147999999999995, 0.07483608487888714, 0.17022435313432682, 0.10513370582263332, 1.2428805879890474, 0.9801326330655459, 1.323016568906074, 0.3789822555212843, 0.9700128514612577, 0.2979359142500279, 1.2429810119225475, 0.9825053058380907, 1.3285820381143199, 0.15664944178238732, 0.14990974565663218, 0.08891696617581848, 0.15799999999999992, 0.6823999999999999, 1.138002114427595, 3.7152000000000003, 9.5496, 2.7877, 8.085318979351786, 6.2702644806469285, 7.936206267958993, 14.111870000000001, 92.70797366, 8.337194300000002, 77.69912348, 51.61267835999998, 90.85490161999998], dtype=np.float32)

# fifo
# test_input = np.empty((INPUT_NODES, ), dtype=np.uint32)
# for i in range(INPUT_NODES):
#     test_input[i] = i

# test_input = Fxp(test_input, signed=True, n_word=32, n_frac=15)

# --------------- TIMING ---------------- #
start = time.time()
pred_output = get_fpga_output(test_input)
pred_output = get_fpga_output(test_input)
pred_output = get_fpga_output(test_input)
pred_output = get_fpga_output(test_input)
pred_output = get_fpga_output(test_input)
end = time.time()
print(f"Time elapsed: {(end-start)*1000}ms")
# --------------- TIMING ---------------- #
print(f"Predicted output: {pred_output}")

# # --------------- POWER -------------- #
# recorder.reset()
# with recorder.record(0.5):
#     time.sleep(5)
#     for i in range(10000):
#         pred_output = get_fpga_output(test_input)
#     time.sleep(5)
    
# %matplotlib inline
# recorder.frame['PSINT_FP_power'].plot()
# # --------------- POWER -------------- #

Output buffer: [9.9915612e-01 8.4390875e-04]
Output buffer: [9.9915612e-01 8.4390875e-04]
Output buffer: [9.9915612e-01 8.4390875e-04]
Output buffer: [9.9915612e-01 8.4390875e-04]
Output buffer: [9.9915612e-01 8.4390875e-04]
Time elapsed: 17.17972755432129ms
Predicted output: 0


In [ ]:
TEST_CASES_NUMBER = 100

test_case = [-1.881,-0.4526,-0.3723,-2.8934,-2.7372,-1.5675,0.4702,0.8588,0.293,2.0092,1.5058,3.1261,-0.70617,0.13457999999999998,-0.026740000000000007,0.22146,-0.68953,0.69349,0.9696853057048972,0.3891747186033543,0.21922676022785176,1.6422417387217998,1.1598302453807627,1.750303955574574,1.1995689479975713,0.41178724846697234,0.22085153384117576,1.6571066531759506,1.3493175382392388,1.8826822134922294,0.07724738604796175,0.584,0.26160000000000005,0.9640000000000004,1.3879213088832028,2.414500000000001,9.5611,1.9879650558221378,1.0012741300140138,8.310525369386465,6.895300000000001,9.277740128291546,142.58940309999997,9.55455372,2.5087386599999997,140.21699758,116.27865811999999,204.18593998000003,-0.2325,-1.421,-3.3174,-1.1989,-0.8712,-4.0308,2.6707,1.0617,0.3773,2.2922,1.5559,5.003,1.1112499999999998,-0.23537,-1.4192500000000001,0.12972999999999996,0.021850000000000026,-0.20894,1.1861467946675066,0.8333006792868947,1.4077593851578472,1.168492746276159,0.6620733090073939,2.7028745162141727,1.6253678909711486,0.8659036083768216,1.9990140192104706,1.1756722209017272,0.6624337619717159,2.7109383198442565,0.3152169063568506,0.16240089454894552,0.4323703227726987,1.2972999999999997,0.2184999999999997,2.0894000000000004,11.6195,5.2051,14.192499999999999,5.2649,3.6338999999999997,14.750642304060833,261.3412573,53.805937799999995,394.77145757999995,83.81133819999998,28.567410179999996,373.08061576,0.0825,-1.4618,-0.5017,-1.9811,-1.0198,-2.4307,0.3409,-0.6692,-0.0512,2.2315,1.2419,3.0129,0.19999000000000003,-1.0219699999999998,-0.27885000000000004,-0.13111,-0.025520000000000008,-0.01863999999999999,0.08846435949013591,0.2805278775808208,0.13837650992852799,1.419013373051854,0.6888917226966804,1.8045846199056446,0.21868228780584859,1.0597728865186162,0.31129629133672637,1.4250574672622855,0.6893642549479919,1.8046808859186159,0.11911502152392456,0.26649999999999974,0.07629999999999981,0.9624944302086709,0.2552000000000001,0.18639999999999968,1.9999000000000002,10.2197,2.7885,9.57325775675063,4.113802509600851,12.004874654625707,4.427916360000001,108.41257372,8.736046019999998,102.86850826000004,25.024911640000003,164.48787469999996,0.2504,-1.2982,-0.3264,-1.1931,-4.3889,-2.0722,0.4963,-0.7664,-0.0669,0.7849,1.4671,1.573,0.32767,-0.99225,-0.20748000000000003,-0.22567000000000004,-0.5340499999999999,-0.27944,0.06839823170228891,0.19311327375403275,0.08823032131869406,0.6317148344783428,1.5652590656182126,1.0608605357915808,0.3347326500358159,1.0108673498535798,0.22546072828765545,0.6708133727051064,1.6538577160687071,1.0970468495009682,0.027691270066245922,0.13940961902096582,0.012799999999999923,0.3541000000000002,2.9924153977435344,1.0157961331654164,3.2767,9.9225,2.0748,3.544643606752654,6.881023042826953,5.945393814570495,11.005131919999998,100.35551747999999,4.69410644,25.108569899999996,160.96809137999998,64.82950285999999,-0.3839,-1.5871,-2.0531,-1.1302,-2.2099,-3.161,2.6871,0.7009,0.1417,2.9976,0.3731,4.5297,1.0667799999999996,-0.5610899999999999,-0.9286800000000002,0.2780599999999999,-0.6378200000000002,0.7314700000000001,1.125139225873847,0.7289393149638728,0.8319818373017527,1.5004041350249604,0.8435964281574454,2.256994812599267,1.5504702015840226,0.9198774445544362,1.2468521644525465,1.5259521394853772,1.0575770827698567,2.372566952690693,1.4782467644050301,1.6068032191205779,1.0700518971109871,1.5110580480030944,1.3659215366505615,4.9749618761912044,10.667800000000002,5.6109,9.2868,8.327780058669001,6.3782,11.287873201171056,195.94253122000003,63.11334169999999,132.43779024,131.00711109999997,78.1802808,339.5621876999999,-0.5724,-1.3134,-0.3908,-2.1879,-2.0456,-3.9684,-0.1023,-0.4636,-0.0087,2.0607,2.6272,2.9798,-0.37109,-0.9314499999999999,-0.23227000000000003,0.38883,0.11399000000000008,-0.16910999999999995,0.16956159618262623,0.2668404176656902,0.1300113306600621,1.2226106584272853,1.5562973343484208,2.0359707279084343,0.40799377813883386,0.9689184232947581,0.2661809516099903,1.2829519051780547,1.5604663126770792,2.042981888563871,0.10056798788093248,0.06089999999999929,0.22924384090363764,0.9429979046874913,1.1399000000000004,1.6910999999999996,3.7109000000000005,9.314499999999999,2.3226999999999998,7.512427206498051,8.54451018867302,10.449427688471856,15.715058000000006,90.32195507999998,6.313695439999999,92.90495784000001,146.78149130000003,213.1560325,0.3284,-0.895,-0.9646,-1.1937,-0.7522,-3.5094,0.6397,0.6248,-0.08,1.0123,2.7672,3.2457,0.4983000000000001,-0.12865,-0.50364,0.1052,0.33947,0.396,0.1031943409301111,0.597695116677391,0.2525798455934281,0.769017077833776,0.9643306248896174,2.3844749606569575,0.5088732278279139,0.6113839014890725,0.5634268612694997,0.776179300162018,1.0223372413249945,2.417134013247921,0.0735203504756399,0.5206868216286866,0.29019837821196903,1.0519999999999998,1.532593741037417,1.8044,4.9830000000000005,2.8189461601425125,5.0364,3.466353746092081,4.18768785860564,10.558868576560881,25.561444680000005,23.132146479999996,29.561083160000003,36.60043772,65.7427228,301.5955715799999,-0.0081,-0.9615,-1.1391,-1.1703,-1.6242,-3.5939,1.394,0.4115,0.4277,1.3665,3.5196,4.5833,0.63713,-0.40476,-0.4954500000000001,0.22953,0.39067999999999997,0.0676600000000001,0.4809180825254962,0.5514167030477042,0.5393327511842758,0.81140757828603,1.2755515644614293,2.9095314836584945,0.7982586291422097,0.6840256120935824,0.7323595558194077,0.8432474601206932,1.3340399754130308,2.9103180805540827,0.3609450600085027,0.23819999999999997,0.3730049372122129,1.6593790134228035,2.413,0.6766000000000005,6.3713,4.0476,4.9545,3.176509409119602,5.791812458036306,16.357069091663725,54.14499364,31.6144544,44.769174080000006,42.30395623999999,99.52596042000002,461.9797154,0.1385,-0.8965,-1.3005,-1.0628,-1.1833,-3.2771,0.9313,0.5249,0.0231,1.6067,3.4775,2.9829,0.47473,-0.33043000000000006,-0.58446,-0.11306000000000001,0.65027,-0.45660000000000006,0.3180992991190015,0.495918263527368,0.4287564558114548,0.8639342882418778,1.4936772342443998,2.283279813338698,0.5714505551664115,0.5959185422522109,0.7248624628162228,0.8713007620793178,1.629086478060634,2.328486690105829,0.350873248207561,0.25076025454443907,0.5234058247938507,1.1306,1.1391,2.734807874152212,4.7473,3.3042999999999996,5.8446,4.272914727209871,8.222596387563756,11.389783327240034,28.661577539999996,24.1782779,46.5483688,43.26144789999999,154.4874657,293.94080682,-3.474,-0.508,-0.4607,-2.5187,-1.608,-3.6564,0.324,0.787,0.1934,2.8147,2.958,2.7862,-1.2921999999999998,0.23599,-0.14644,0.16158000000000003,0.4418,0.15113,1.1209164607587847,0.4881840952960266,0.18721248462642656,1.8214188171862067,1.3102978241605991,2.011021014833012,1.7106240241502515,0.5422314920769542,0.2376829568984701,1.828571738816938,1.382775335331087,2.0166917957387542,1.676473878878279,0.20138863165772675,0.16839847255724746,1.383091110548865,1.7831582311841425,1.5112999999999994,12.922,2.6330158163290682,1.4644,11.401412112099129,7.574857422589566,9.92537313658572,256.72095648000004,18.278987560000004,3.9390591599999993,170.12790910000004,119.58638787999999,208.46233000000004,-2.8592,-0.7762,-0.4939,-2.9599,-2.2797,-2.3221,0.2762,0.485,0.3227,2.0149,2.8116,3.315,-1.54238,-0.1338,-0.14392,-0.79024,-0.8121499999999999,-0.36429999999999996,1.339385299157789,0.46524101280948993,0.2982279792373613,1.7402520427800108,1.539637055445211,1.8855661834048678,2.0427650486534175,0.4840987915704809,0.3311387835938279,1.9112708939341907,1.7407095923789242,1.9204359718563906,2.4150756730270326,0.6211302390944226,0.2509400362595457,0.44460000000000033,2.1363512487132,2.760422377812818,15.4238,2.5532,1.973,10.061113806003782,8.140533301420513,8.843024008972906,385.421966,15.872119119999997,8.464657520000001,213.97061896000008,196.57181158000003,213.73302510000002,0.0753,-1.3681,-0.2601,-1.0978,-1.2126,-2.3071,0.4905,-0.6232,0.0347,1.0683,2.4838,2.7623,0.29801,-1.01523,-0.07595,0.018990000000000007,0.18183000000000002,-0.0017700000000000118,0.13907529219814713,0.23933500809534738,0.0882241491883033,0.5852403548115936,1.219809527795221,1.4764879071973465,0.3288645572268316,1.0430595376103897,0.116412641066166,0.5855483694794138,1.2332872467515426,1.4764889681267517,0.32481568214578443,0.10714038036253971,0.12510000000000004,0.18990000000000007,1.8182999999999996,0.017700000000000715,2.9801,10.1523,0.7595000000000001,3.0935884060936796,5.230467029841824,7.320596883247908,10.125530860000003,106.06366484,0.97384028,18.241090780000007,91.09794949999997,116.16538834000002,-0.479,-1.2179,-0.4232,-1.8137,-1.6443,-2.4633,-0.25,-0.6336,-0.0521,1.7223,1.2101,1.5145,-0.37152,-0.9549599999999998,-0.27877,0.01579999999999997,-0.06824000000000001,-0.12147999999999995,0.07483608487888714,0.17022435313432682,0.10513370582263332,1.2428805879890474,0.9801326330655459,1.323016568906074,0.3789822555212843,0.9700128514612577,0.2979359142500279,1.2429810119225475,0.9825053058380907,1.3285820381143199,0.15664944178238732,0.14990974565663218,0.08891696617581848,0.15799999999999992,0.6823999999999999,1.138002114427595,3.7152000000000003,9.5496,2.7877,8.085318979351786,6.2702644806469285,7.936206267958993,14.111870000000001,92.70797366,8.337194300000002,77.69912348,51.61267835999998,90.85490161999998,-0.5839,-1.1637,-2.0479,-1.57,-1.8759,-3.417,2.6841,0.7465,0.4193,1.1029,3.7441,2.9877,1.04314,-0.14406999999999998,-0.81569,-0.20795,0.25734999999999997,-0.6546900000000001,1.3085246189506716,0.6689043190920508,0.9038008790104156,0.7892268865795184,1.523121567209919,1.8893442616156537,1.6734329200777665,0.6842434895561668,1.2174589130644204,0.8161631460682355,1.544709788601082,1.999560135379779,0.7291229082878451,0.5159293407807645,0.22056728065220582,0.813226199637724,2.0614999999999997,2.190808792663823,12.526599999999998,3.2489,8.521500000000001,3.2578479204457387,6.459144293442198,9.761366845129707,272.88379366,29.7249415,143.68580018000006,35.873904419999995,124.74275879999995,225.52221616000006,0.1081,-0.9113,-1.6381,-0.6062,-1.2926,-3.5149,2.2097,0.3949,-0.0609,1.2932,1.5655,2.8728,0.79179,-0.37103,-0.7532800000000001,0.24745000000000003,-0.10926,-0.7122399999999999,0.6712409812429513,0.42832216391403327,0.5171834506246309,0.5849528771619129,0.7734565614176403,2.1993288077047506,1.0380249799499048,0.5666772776457515,0.9137338124421138,0.6351388596204769,0.7811355836216911,2.3117813525504523,0.43409999999999993,0.8180999999999998,0.6058000000000003,0.4819316872966913,0.30559999999999987,0.8456000000000001,7.9179,3.7102999999999997,7.5328,3.3725325857372463,3.396689598087307,14.54741808186182,85.31558456,23.2739637,70.30050874000001,23.850471799999998,31.15222306,292.93846166000003,0.1763,-1.6687,-0.5146,-2.0588,-2.0008,-1.811,0.5524,-0.7889,-0.0059,2.4596,1.7194,2.4131,0.32317,-1.0817599999999998,-0.26666,0.04196999999999997,-0.03386,0.2836200000000001,0.1016304880436968,0.2617380262781853,0.1524922961988572,1.6262411192993491,1.113112213750258,1.4737455823852366,0.3387736781392557,1.1129741650191167,0.3071830984933904,1.626782609631662,1.1136270919836675,1.5007885747166387,0.15723130112824465,0.006199999999999761,0.13498915791262361,0.41969999999999974,0.33859999999999957,1.1531994060980517,3.2317,10.8176,2.6666,10.781953639308018,6.84054035546574,9.495896534082684,11.119088819999998,120.4458287,8.417098579999998,137.08147580000002,74.01870616,124.69321963999997,0.0143,-1.302,-0.3767,-1.8183,-0.9805,-2.4903,0.2813,-0.7159,-0.0543,1.412,1.0291,1.5274,0.17365999999999998,-0.9818199999999999,-0.20943,0.020009999999999972,-0.09570000000000001,0.022710000000000053,0.08844900451672703,0.20136576074397558,0.10572865316459866,1.285289198157364,0.6391920384360245,1.3750274873252535,0.19488720327409903,1.0022567944394292,0.23460492961572654,1.2854449513689803,0.6463164488081671,1.3752150140977955,0.10856349148763487,0.1461137865733025,0.06729999999999992,0.20009999999999994,0.3429729857546432,0.22710000000000008,1.7366,9.818200000000001,2.0943,8.604034193523429,4.0866,8.767195739459149,3.44420538,98.43713204000002,4.94728454,85.14972076,29.694321879999997,100.53268007999999,-0.1951,-1.3717,-1.4507,-1.3414,-0.4929,-2.795,1.7499,0.6631,0.4072,0.2294,3.2557,2.9384,0.49156000000000005,-0.35346,-0.5419,-0.34758,1.31872,0.13627,0.5193838988647993,0.6221673716935018,0.6900701848362962,0.43071451751711365,1.3353334593276691,1.9154745913480555,0.7151159822014889,0.7155600673598268,0.877412371693037,0.553468022563183,1.8767359659792318,1.9203157091999221,0.6813137317579621,0.5457836759369741,0.6932865021134602,0.8675694319651986,0.7244262706263781,1.3627000000000002,4.9156,3.5345999999999997,6.316600000000001,3.4757999999999996,13.187199999999999,10.09648697721908,46.13583326,32.799933100000004,73.12512178000003,23.633059899999992,303.9115845399999,195.91901792000004,-0.585,-1.4627,-0.3329,-1.0906,-1.7691,-2.4611,-0.0928,-0.4911,-0.1402,1.477,1.3118,2.0032,-0.35306000000000004,-1.0253999999999999,-0.24702999999999994,-0.11239999999999999,0.004280000000000039,0.15302999999999997,0.15181654850509543,0.3259126109864422,0.06946232144119573,0.7050496776823603,1.1169735160692038,1.1889416142519362,0.3843170930364664,1.0759480424258412,0.25661027843794565,0.7139529452281852,1.1169817160544753,1.1987494913450434,0.27211212743508695,0.0025999999999992696,0.18569999999999998,1.1239999999999997,0.042800000000000615,0.6663000000000006,3.5305999999999997,10.254,2.4703,2.7787944674275797,8.4484,4.494270487839776,13.99861408,110.45547088000001,6.360875039999999,28.868109920000002,98.07105490000001,73.24290403999998,-0.1421,-1.0693,-1.8497,-0.8227,-1.6565,-3.2034,1.901,0.9239,0.1581,0.6346,1.9772,2.6598,0.65979,-0.37370000000000003,-0.7341200000000001,-0.08852,0.06466999999999999,-0.37833999999999995,0.7885828548098164,0.6086413558081639,0.7433858080431721,0.46033835990497257,0.9596610412536294,2.0385251973914866,1.0281953914504773,0.7142100461348888,1.0447749202579473,0.4687720085499987,0.9618375762050473,2.073336956695655,1.048685761839726,0.9489753136427618,0.3196363311294705,0.3820868617279414,0.6466999999999998,1.4813152592434278,6.9171000000000005,4.2808,7.3412,2.396374117306163,4.010389996260126,11.553173802111468,98.54856656000001,41.65000832,106.5196185,12.229618139999994,47.484279240000006,226.51332369999997,0.1629,-1.3547,-0.4324,-2.1413,-1.4101,-2.1637,0.3678,-0.838,-0.0827,1.9284,1.8993,1.525,0.28346000000000005,-1.00383,-0.24057,-0.2516999999999999,0.042859999999999975,-0.17152999999999996,0.0762901854762459,0.16610064449001993,0.11736081160251065,1.392231266708229,0.9898587537623739,1.3898454713024753,0.29354686848951395,1.0174792838185944,0.26767047838713925,1.4148006184618382,0.9907862191209564,1.400390293810979,0.11855811625471889,0.1933000000000007,0.07510000000000017,0.7920822669666947,0.4286,1.1211347999875265,2.8346,10.0383,2.4057000000000004,9.031019174620155,5.997135730834108,8.423847059999185,8.430117260000001,102.16562053999999,6.478890500000001,105.1773685,49.58242008,102.22417259999996,0.0442,-1.1892,-0.738,-0.8166,-0.8198,-3.9322,0.9132,0.4175,0.1113,1.1802,1.8275,3.8335,0.55487,-0.29856999999999995,-0.34956,0.29664,0.25806,0.20408999999999997,0.2803422624222042,0.5946007568949101,0.24948222862560773,0.6605502209521997,0.8631841648223164,2.813131090244463,0.6216691250174806,0.6653526170385144,0.4294573040477947,0.7241007416098951,0.9009338854766203,2.820524642508907,0.6638999999999999,0.1986545366958112,0.49760503540336665,0.4176352643490472,0.8228909635043096,1.7655836187513088,5.5487,3.409255599841434,3.4956,3.689963107863464,4.718474945592373,15.733409983700199,34.9380425,28.0063013,16.03150626,30.71204366,44.400925980000004,413.2704256,0.1808,-1.5787,-0.4273,-1.9962,-1.3794,-2.3505,0.4052,-0.7441,-0.0591,2.2407,1.7584,3.2885,0.31164000000000003,-1.01173,-0.27279,-0.0020299999999999875,0.28964,0.18472000000000005,0.06868390204407435,0.25461598162723403,0.12943581768583223,1.5181334487125957,0.8708791537291497,1.83342323471696,0.3191190498857754,1.0432769962958064,0.3019404163075887,1.5181348059378654,0.917780927019079,1.8427051408187909,0.014428519473574512,0.23489999999999966,0.07253077675753697,0.020299999999999763,0.8183971845141336,1.5279999999999996,3.1164000000000005,10.1173,2.7279,10.184582172796867,4.2906203139382235,12.350245209306053,10.036780500000004,105.62881320000002,8.34054508,119.34872213999998,49.9389372,172.65157772,-0.2412,-1.7505,-1.8712,-0.8475,-2.5156,-3.0067,2.8191,0.6917,0.3543,1.6741,1.5252,4.3563,0.8534599999999999,-0.64227,-0.8237099999999998,0.3411100000000001,-0.6566799999999999,0.47467000000000004,0.9886276834076618,0.7651690833404078,0.8980343138767026,0.8494581984418068,1.1721792310052248,2.3137796464011,1.3060546190722653,0.9989967362309048,1.2185909046927932,0.9153880395766595,1.343589510229966,2.3619669474825424,1.526421985763778,0.13789794656775492,0.26447055455577284,0.6328894631410893,1.5458940612319254,2.227648299132032,8.5346,6.422700000000001,8.7357,4.0515,6.566800000000001,13.154453513368551,157.54182365999998,90.48192824000002,146.3293251,55.92189087999999,121.17501724000002,293.35927054000007,0.2568,-1.3467,-0.5376,-2.2925,-1.374,-2.2286,0.444,-0.7621,-0.0235,2.1694,0.8558,2.2661,0.33363,-1.02317,-0.2192,-0.33377,-0.004379999999999995,-0.12553000000000006,0.05260756694621032,0.1702910863785888,0.14627477567919903,1.646293841360041,0.698704135954554,1.4386600363185182,0.3377521768989802,1.0372443795943171,0.26352409756984274,1.6797874344690162,0.6987178643773179,1.4441261998177306,0.043500000000000094,0.09450000000000003,0.13954459687635992,0.5188389766780789,0.04380000000000028,0.029100000000000348,3.3362999999999996,10.2317,2.192,11.197535946684834,3.970201693210539,9.119666662455032,11.270221619999997,106.14210272000001,5.903527520000002,146.96907434000002,24.6120164,105.06333649999995,-0.1248,-1.0493,-1.874,-1.5623,-0.6656,-2.9645,1.8176,0.6597,0.1316,1.3241,1.3829,4.0197,0.64471,-0.22011000000000003,-0.82643,-0.11993000000000001,0.4509,0.033220000000000006,0.6956006878231218,0.5170916059075026,0.7323159796836335,0.863085133749852,0.6564565926243715,2.026802889676251,0.9484256960879962,0.5619894491892174,1.1042070634622838,0.8713777326739535,0.7963956730168742,2.027075115036441,0.6173401231790058,0.4184543532857148,0.23410605629697376,1.1993,0.13660000000000005,0.33219999999999983,6.4471,3.3353,8.2643,4.469053615714458,4.509,10.669893551517735,84.53982186,23.776140700000003,120.44381584000003,39.45502734,41.88717368,212.70331374,-0.4391,-1.1643,-0.3921,-1.4836,-3.0471,-2.1668,-0.278,-0.5907,-0.1229,2.0852,1.3749,1.777,-0.3579,-0.9372999999999998,-0.24394,-0.13124,0.023780000000000016,0.15543,0.05984829153785427,0.18807674497396001,0.09360358112807438,1.2206112003418614,1.1589630932864083,1.3263832628995287,0.3628694365746446,0.9559833429511206,0.2612821348657424,1.2276463823104762,1.159207030689514,1.3354591139379746,0.08961074698632528,0.23512277601979406,0.13179834479111377,0.26902697230630174,0.2378,1.5543,3.5789999999999997,9.373,2.4394,6.2874929231110634,7.416399999999999,7.185384681974877,13.013464719999998,89.90137451999999,6.41221166,87.28831455999999,94.71781589999998,95.00887962000003,-0.5794,-1.3507,-0.3787,-0.7742,-3.5956,-2.989,-0.1587,-0.434,-0.0592,1.6546,2.4361,2.1926,-0.39463,-0.9730700000000001,-0.1986,0.20329000000000003,-0.0006999999999999174,-0.38292,0.13345846582364118,0.2867055878423021,0.10799793516544658,0.7166409930362622,1.6453368706742095,1.5596852642760974,0.4165861243488554,1.0144285677168206,0.2260652870300967,0.7449168658313491,1.6453370195798793,1.606003066622228,0.20940092355370385,0.36717290289356713,0.13260872213224922,0.6905000000000001,0.006999999999999229,0.36019999999999963,3.9463,9.730699999999999,1.9860000000000002,3.4276605958438164,7.181871091598876,8.639806604930307,16.488196040000002,101.10799219999996,4.620512980000001,30.049793179999995,153.44081789999998,136.35855083999996,-3.4747,-1.0032,-0.1352,-3.147,-1.9176,-2.1646,0.5585,0.766,0.4185,3.3316,2.345,2.8539,-1.0261,-0.07763,0.1358,0.45594,0.37133000000000005,0.51725,1.6744314133460347,0.5736883806562584,0.19956442568754584,1.750593720541691,1.3220650370159557,1.8241050629007092,1.9638232527394108,0.578916898181423,0.24138682648396534,1.8089941011512447,1.3732231905265802,1.8960239563359953,2.028466631873654,0.28679163688314324,0.17785377083099324,2.1205971991346946,1.021830990365206,0.37529999999999974,10.261000000000001,3.552974966814677,1.358,10.798015514537722,7.610275861155934,11.587045279495896,270.74877692,17.925409039999998,3.863688880000001,177.32258179999994,106.79298000000003,193.19314531999996,-2.6903,-0.6811,-0.1638,-2.3256,-2.1993,-3.7907,-0.0142,0.9913,0.3297,3.1981,1.7635,2.1706,-1.0640200000000002,0.2217,0.06561999999999998,-0.19206000000000006,0.057379999999999966,-0.8475500000000002,0.8783444162741629,0.6820551619920489,0.17368665348840134,2.0252274460909323,1.1649276602433303,1.7849507563235465,1.3797200708839457,0.7171820786941069,0.1856691627600017,2.034313951188459,1.1663399667335421,1.9759529865358638,0.3646801494126593,0.3633472573680043,0.16999999999999998,1.8748000000000002,0.5737999999999999,2.501891456462609,10.6402,3.4906866789448676,0.9794658193643808,9.733852422262558,6.619999999999999,8.671733710168883,181.18741372,31.226489280000003,1.9534011199999994,210.52345229999995,90.09426911999999,236.29342650000004,-0.3502,-1.3555,-0.3679,-1.2247,-1.0731,-1.9085,-0.1592,-0.746,-0.0759,1.286,1.0751,1.2645,-0.27971,-0.95275,-0.20546000000000003,0.1798,-0.04800000000000002,-0.15772000000000003,0.05542944073324212,0.18012331470412152,0.08779456930812976,0.7610088172945173,0.5729837554416355,0.955574242850863,0.2851492714351555,0.9696272330127698,0.22343164055254125,0.7819606511839327,0.5749907686215493,0.9685028301455809,0.007106066355297541,0.1176968784849533,0.11172249013512649,0.4939039749810465,0.4800000000000002,1.2489104231895194,2.7971000000000004,9.5275,2.0545999999999998,3.4646508768909086,2.7442,4.53829268567716,7.997933200000003,92.43560612,4.620256299999999,32.64725811999999,20.411236019999997,49.2471702,-3.4703,-0.6736,-0.7225,-2.3548,-1.7125,-2.3183,0.331,0.5219,0.263,1.6808,1.9169,2.6616,-1.13353,-0.04586,-0.20819000000000001,-0.37012999999999996,-0.46955,0.09122,1.3950702520303413,0.46102024510860695,0.33774422112006597,1.3838942488860917,1.0686536007986873,1.801578358440176,1.797529212280012,0.4632955924677031,0.39675462820236895,1.4325360410823875,1.1672607767761238,1.8038862685879062,0.24073395586525495,0.4586,0.4276383297137348,1.9068911016151828,1.3175,0.9121999999999999,12.898500000000002,2.6770568422317016,2.5799000000000003,6.201747811547406,5.1137157837949365,8.422006322880017,308.98572762000003,11.958999699999996,13.36580756,111.77054153999998,80.0166493,165.1215413,-1.0392,-1.1844,-0.5105,-1.8823,-2.134,-2.384,-0.1869,0.2356,0.1087,1.9394,1.8805,2.7039,-0.47028,-0.75315,-0.21688999999999997,0.23861,-0.19251000000000001,-0.07734999999999997,0.21804367819315465,0.405322400688637,0.15823657257410498,1.2789249418554633,1.3872683946879205,1.487438485618817,0.5183689072465669,0.8552901092611793,0.2684773454129789,1.300993366239813,1.4005619225867878,1.48944831095275,0.5470160562782274,0.0486999999999993,0.2510172311206,2.3861,1.9250999999999987,0.7734999999999996,4.7028,7.5315,2.1689000000000003,6.3333,11.6871,8.370703720605526,24.85133812,64.93899052,6.279838900000001,107.53126800000001,168.22584315999995,118.89536780000002,-0.3503,-1.4981,-1.539,-1.0965,-3.1782,-4.0756,2.4887,0.7016,0.4827,1.1847,2.4688,3.3477,0.72664,-0.5056499999999999,-0.5860099999999999,0.22749000000000003,0.024849999999999994,-0.80175,0.7859719819433769,0.7669935504943963,0.6903265784395093,0.7487177645147736,1.797700332230041,2.5879414642723275,1.0704006941328092,0.9186735160001076,0.9055156017430069,0.7825151698210073,1.797872077484936,2.709288483163061,0.507043106454662,0.7474318236431523,0.2628190925369163,0.5047000000000001,0.24849999999999994,2.357491781975706,7.266400000000001,5.4019,6.3571,4.083253877180512,8.853424103908662,12.459597070046728,104.73913176000002,69.57240938000001,78.37467146,33.331445599999995,174.04877447999996,415.72676049999995,-0.3816,-1.2576,-0.6967,-0.4791,-1.0498,-1.3028,-0.0309,-0.576,-0.2148,1.3171,0.5855,1.1945,-0.22601,-0.9606799999999998,-0.34181000000000006,0.17784000000000003,0.06583000000000001,-0.07105999999999998,0.10452735957633294,0.21951754736239198,0.13057964198143598,0.5396207746186205,0.5015595239849404,0.745341421363391,0.24901102184441556,0.985441026140073,0.3659031552200664,0.5681704374569306,0.5058611914349628,0.7487211483589868,0.239050152137295,0.3889703118547738,0.3605984327961308,0.021999999999999797,0.5880497701336187,0.5128000000000001,2.2601,9.6068,3.4181,3.151924512232654,2.730970428368671,4.578319612053817,5.76176658,95.08474384,12.66759556,17.72247758,13.341534819999996,28.41312599999999,0.1126,-1.1709,-0.431,-1.4546,-1.5062,-1.8762,0.2863,-0.8022,-0.1252,1.3993,1.2139,2.0311,0.18936999999999998,-0.9749299999999999,-0.26825,-0.12883,-0.07918000000000003,-0.05194999999999998,0.0537651941315197,0.13169018224605808,0.10912044950420613,0.9459608692224009,0.6933641810765826,1.2201974538983436,0.19685449702762697,0.9837839239385852,0.2895951225417997,0.9546932151220098,0.6978705897227652,1.221302840822046,0.1379892204147349,0.2113064664120263,0.03770000000000007,1.1617767287673315,0.13079999999999997,0.5194999999999999,1.8937,9.7493,2.6825,5.989152482822146,3.9788755471295243,7.324681385936264,3.7441975399999996,96.00700214,7.79188052,47.08194303999999,24.673195939999996,76.58782598,-0.3411,-1.2322,-0.4543,-1.7773,-2.32,-1.2712,0.0491,-0.5797,-0.1144,2.0919,1.182,1.7531,-0.21077000000000004,-0.97367,-0.31201999999999996,0.11549000000000001,-0.08609000000000001,-0.021089999999999998,0.11004824442034503,0.19053995407787838,0.11399941929676659,1.1678587940757221,1.1604771022730263,1.0073012185538148,0.23777007591368599,0.9921384696704387,0.33219323894384123,1.1735553259220461,1.1636660143701025,1.0075219764352539,0.20840610443984292,0.2555659756656992,0.12979999999999992,0.24918306093674128,0.8609000000000002,0.1997000000000002,2.1077000000000004,9.736699999999999,3.1202,7.05286720488292,6.108890332294234,5.80447498543055,5.129013740000002,96.69321443999998,10.39386544,71.91933060000001,69.06084886000001,50.7972061,-2.8252,-0.7186,-0.3333,-3.3932,-2.2012,-3.5837,0.253,0.9848,0.3274,2.8313,1.3179,2.8039,-0.9637100000000001,0.05989,0.04048,-0.10092000000000004,-0.52113,-0.13122000000000003,0.9566338708722372,0.5766076403413329,0.21713482355439903,2.191688538912407,1.0784878794404693,1.79481726356752,1.3578973919262087,0.5797095678009808,0.22087589728170887,2.1940108244947196,1.1977948835255559,1.7996076511284342,1.5157782802632376,0.18434974897467207,0.056800000000000045,0.679871490672996,0.5852999999999997,1.3122,9.6371,3.0878623435160137,1.2523407012415635,13.412104653490074,5.2113,9.679242113661953,165.33820068,19.110277959999998,2.52285274,255.94564284000003,85.48574103999997,164.17243870000001,-3.6037,-0.6723,-0.7928,-2.9671,-1.9061,-2.0809,0.527,1.0794,0.1728,2.214,1.1056,3.1157,-1.0665,0.15233999999999998,-0.26586,-0.27168,-0.33348,0.35821,1.404825115094402,0.5889605032597687,0.30465837654658373,1.6452260068452602,0.9698687919507464,1.9794716519566526,1.7637901955731583,0.6083436117853133,0.40434918820247434,1.6675067124302678,1.0255995241808569,2.0116218394618803,0.8538479246988843,0.2549999999999999,0.3127499357995114,1.660236892919301,1.3632968572174597,0.1884999999999999,12.720600000000001,3.5474625539663234,2.6586000000000003,7.94820033337042,4.580346269563142,12.390074158068497,293.32573737999996,19.69698378,14.58899028,147.18410253999997,62.90992639999999,208.76460758,-3.0317,-0.5063,-0.2791,-3.6102,-2.4052,-3.1683,-0.0685,1.5722,0.7681,4.3636,1.6286,2.8809,-1.3073799999999998,0.70189,0.1648,0.35519999999999996,-0.16254000000000005,-0.11573999999999995,1.1169556587438911,0.8205059639636996,0.3432162233927761,2.4471176743262677,1.2427878694290513,1.9547781332928809,1.7195442442693936,1.0797590513628492,0.38073142239641844,2.4727620087667153,1.2533718283095403,1.9582015468281093,1.2553303571256542,1.626716082650359,0.07825360034540554,3.2988,1.6254000000000004,1.1573999999999995,13.0738,7.0188999999999995,2.1586,13.55244345179529,5.7142130585601825,9.26374651397647,280.80184940000004,84.95999849999998,10.935549779999999,317.4769903199999,82.4859392,198.13943499999993,-0.504,-1.2363,-0.3711,-1.3822,-1.1124,-1.61,-0.0695,-0.6684,-0.0463,1.1471,1.2014,1.3203,-0.30921,-0.9596699999999998,-0.20021,0.19946,-0.06296000000000002,-0.15924000000000005,0.11117760970627133,0.2245551293112673,0.11568818824754755,0.7615349502156812,0.8258791039855652,0.9239027797338852,0.3285898431175255,0.9855919617164093,0.2312310554402241,0.7872228223317717,0.8282754710843488,0.9375253191247691,0.21778150428115156,0.13465007107215407,0.10398157354274981,0.4934000000000003,0.6296000000000004,1.1672094692797215,3.0921,9.5967,2.0021,3.6404701194199345,7.713000000000001,4.552619110893434,10.351801099999998,95.78605731999998,4.855259859999999,33.09692496000001,64.24539538000002,51.175087959999985,-0.6358,-1.3429,-0.3606,-1.7705,-2.2548,-2.2089,-0.0193,-0.68,-0.0645,2.3852,2.9424,2.4132,-0.30262,-0.95684,-0.20509,-0.13247999999999988,0.07274000000000003,0.03424999999999998,0.1591198403719662,0.22655542015144992,0.0917185744546872,1.3434025649819192,1.3840138960285044,1.4636993251689363,0.3419034776073505,0.9832955527205438,0.2246646500898617,1.3499190353498984,1.385924085944104,1.464099988730278,0.05512758472998417,0.25093985363370236,0.04970000000000008,0.28721554199593513,0.49525295224960464,0.3424999999999996,3.0262,9.5684,2.0509,8.914281033935792,7.8597412701238465,9.709112019386556,11.174155119999998,94.37625898000002,4.6280407,93.4749485,98.52650925999997,109.65347418,-0.6069,-1.3605,-0.3696,-1.438,-0.6204,-2.3576,-0.2601,-0.4551,-0.0715,1.0055,1.3565,2.2504,-0.44089999999999996,-0.96158,-0.22363999999999998,-0.16964999999999997,0.31499,0.4714899999999999,0.13158502954363768,0.23867635743826832,0.10003630540958618,0.7760521068201541,0.5858893981802368,1.4241168389215824,0.46011675692154486,0.9907585477804367,0.24499410605155383,0.7943789995965402,0.6651955253908433,1.5001371907262349,0.13017887405836479,0.2036878912708627,0.07749705734399953,0.2972124294570215,1.4031202061311194,1.5251454188979334,4.409,9.6158,2.2363999999999997,5.1576178209642505,3.5787,8.670908691360752,20.443665779999996,96.3856284,5.513583199999999,34.102943720000006,33.488736200000005,127.31888996,-2.6984,-0.4736,-0.2092,-2.3889,-2.4407,-1.6779,0.4043,0.4727,0.5299,2.3405,1.4543,2.1478,-1.05815,0.01788,0.07484000000000002,-0.07167000000000004,-0.65488,0.21479000000000004,1.1126988867164378,0.34552759310943604,0.2194986842785168,1.8658635030730408,1.1726029232438404,1.3264141747207017,1.5355065076384407,0.3459899015867371,0.23190665794668336,1.867239460005063,1.3430805746491907,1.3436924153242809,0.8899919424566772,0.17880000000000007,0.3733765826041034,0.638818053877926,1.833201343680469,1.5970195780077736,10.5815,1.3810978323315224,1.0478133409929848,10.90463199856096,7.168199999999999,8.130041870734427,228.50882332,6.04441844,3.0802894600000004,183.33598893999996,137.32820784,96.13571396000003,0.2247,-1.5257,-0.3129,-0.2944,-1.5497,-2.9627,0.5469,-0.6762,-0.0535,1.0129,1.2879,3.7915,0.37795,-0.99572,-0.19593,0.32597,-0.19837,0.5968899999999999,0.1110070110398438,0.2991769202328281,0.0868397840854064,0.4352896071582688,0.8183766040766317,2.055742053590382,0.3939146595393474,1.0396947378918486,0.21431218584112288,0.5438138311959342,0.8420753665795004,2.140643142375674,0.19978411522312722,0.4308264556619636,0.0877214561027832,0.8523023594628882,1.5167472855930408,3.201897683780957,3.7795,9.9572,1.9593,3.2596999999999996,3.8632176812395147,8.723915435246228,14.921740080000001,103.8555119,4.21983002,21.483445039999996,43.479715240000004,278.5860742,0.1057,-1.4199,-0.4731,-1.782,-1.1201,-1.8231,0.3767,-0.6929,-0.0827,1.926,1.4434,2.0084,0.19607000000000002,-1.04637,-0.26245,-0.17824999999999996,-0.02489000000000005,-0.12219000000000002,0.08295654344293764,0.2834194172952869,0.12482548818250222,1.312417809426556,0.7704469435983247,1.285476044467574,0.21289723577350644,1.0840741409147254,0.2906224440747824,1.3244672774364794,0.7708488859692281,1.29127032684872,0.11010000000000009,0.16710000000000047,0.09709577534329324,0.07370000000000076,0.24889999999999973,0.3236999999999999,1.9607,10.4637,2.6245000000000003,8.53338459840968,4.360161802768353,8.158384099975763,4.1944949000000005,113.51930719999999,7.677889580000001,89.30204741999998,29.871476860000005,84.16786349999998,0.0241,-1.3075,-0.488,-2.2527,-1.3794,-2.1559,0.2783,-0.8095,-0.1102,1.6894,1.6046,1.6596,0.18153999999999998,-1.06601,-0.25261,-0.17017,-0.17343999999999998,0.042650000000000035,0.06551980158700116,0.17852737297120572,0.12680189627919608,1.3880345082525867,0.851292552769023,1.193723845996217,0.19300159584832452,1.080855838213404,0.28264913408676845,1.3984268393448405,0.8687810103817877,1.194485513934765,0.06440441292192071,0.22510026500637756,0.10150000000000015,0.1386999999999996,0.5078955827353048,0.17289999999999994,1.8154000000000001,10.6601,2.5261,9.66099609817598,4.213316692009637,7.95411358728891,3.5301198800000004,115.44656676,7.19026838,99.23739153999998,41.68735438000001,71.44568047999996,0.1351,-1.2794,-0.4325,-1.8803,-2.0917,-2.2322,0.4559,-0.8107,-0.0853,1.9829,1.5957,3.1362,0.30699,-1.0278200000000002,-0.24259999999999998,-0.02559,-0.023780000000000003,0.18373,0.09096491026764113,0.13923681840662694,0.11157847462660529,1.367321369283754,1.0021558359856015,1.7521752047383845,0.3201835020734204,1.0372081970366411,0.2670290545989331,1.36756081217619,1.0024379322431887,1.7617816723419506,0.07169999999999987,0.1776997009864892,0.20597951072673465,0.25590000000000024,0.23779999999999957,1.5785,3.0699,10.2782,2.426,8.594953757777178,5.108043463788089,10.49987725424227,9.840587200000002,106.7927856,6.54062948,94.28990228000004,51.235081699999995,158.12739982000002,-0.2362,-1.7549,-2.2594,-1.2064,-0.7213,-3.0041,1.759,0.7442,0.281,2.1893,4.5078,3.6503,0.8744100000000001,-0.5450900000000001,-1.01168,0.18250999999999992,0.6534199999999999,0.30612000000000006,0.7629136221748829,0.8304331357189452,0.930106890416365,1.166233122878955,1.479967248826811,2.2323796078624265,1.1604438991179193,0.9933490328177704,1.3742617108833382,1.1804277178209601,1.6177950284260363,2.2532705492239495,0.7204401433474419,0.6507427810313438,0.6519331228902859,0.6006142557184226,1.6079776283290546,3.0612000000000004,8.7441,5.4509,10.116800000000001,6.061368296195402,9.5136,11.461365529198929,126.78310555999997,77.3276205,180.68662210000002,77.39778865999999,197.465215,268.65478834,-0.4802,-1.4223,-0.4782,-1.6842,-3.34,-1.8788,0.2287,-0.727,-0.0903,1.4374,2.8186,1.8946,-0.21221000000000007,-0.99533,-0.24512,-0.12831999999999996,0.28503,0.1422100000000001,0.19406149257387462,0.20860582470295502,0.1238704952763167,1.0539674983603622,1.643530630715473,1.2258546532521708,0.28756381378747914,1.016955357427257,0.27464106393618565,1.0617502107369698,1.6680632586925472,1.2340758951539408,0.22137309097935734,0.16449999999999942,0.12682233167967732,0.9065340110397153,1.8291440395203171,0.4316399244647034,2.1221,9.953299999999999,2.4512,6.178811717304506,8.4977669900582,7.404055150629889,6.608546,101.25753052,6.78608892,59.013026620000005,148.43427692,78.08928008000001,-0.7867,-2.0431,-0.4135,-2.104,-2.5308,-2.1917,-0.1649,-0.5044,-0.1228,1.7697,2.1501,1.1247,-0.41333000000000003,-1.0046799999999998,-0.27837,0.042710000000000026,-0.27816,-0.27841000000000005,0.18454407088822986,0.4706990839166781,0.08603005346970324,1.1674339985198305,1.6497394704619273,1.120675775101791,0.45265682696718496,1.1094771426216947,0.29136064765166897,1.1682149994756958,1.6730251958652624,1.154740802518037,0.12846530255670313,0.5609369319399984,0.050899999999999945,0.42710000000000026,0.8157732596023387,1.6125119212959282,4.1333,10.046800000000001,2.7837,6.785900000000001,15.736,5.5313613626013,18.997660000000003,112.39122284000003,8.1203396,91.35174086,267.63016258000005,78.10869405999999,0.1938,-1.3103,-0.5447,-1.1643,-2.0868,-3.0377,0.606,-0.8107,0.0363,1.6699,1.69,2.4083,0.41816999999999993,-1.0039799999999999,-0.16000999999999999,0.03884000000000003,-0.25933,0.07212999999999992,0.12274659302807552,0.18252767899691272,0.16246398647084836,1.0041445913811418,1.1493596382768971,2.1211540429916917,0.4358128898965702,1.0204372562779154,0.22803014493702362,1.0048954701858297,1.1782527856958371,2.1223800816536134,0.14061739207444252,0.3725999999999994,0.1716537385980267,0.30579999999999985,0.9789677728409732,0.7212999999999994,4.181699999999999,10.0398,1.6000999999999999,5.404175363322797,8.5905,11.1016651646229,18.549714399999996,102.53281709999999,4.1841693599999985,50.612929400000006,109.67492892000001,266.17692423999995,-0.4339,-1.3837,-0.5231,-1.885,-1.5543,-2.1158,-0.0665,-0.7785,-0.1065,2.112,1.7387,1.7394,-0.31272,-1.03539,-0.28827,0.023500000000000097,0.27709,0.006120000000000037,0.11214673245351378,0.21907356960619415,0.12506040180648711,1.4865253674256622,0.9649882459905923,1.381414680535863,0.33222084221192383,1.0583126574883246,0.3142287335683992,1.486711107781199,1.003982660706847,1.3814282370069029,0.29035984889797817,0.35489999999999977,0.22380665921401427,0.23500000000000054,0.4047132954222283,0.06119999999999992,3.1272,10.3539,2.8827,7.211596940954255,6.21646765679024,6.357337514761088,10.592691700000001,109.66588365999998,9.20644562,116.24449752000001,57.26355655999999,104.53125192000002,-3.3753,-0.4663,-0.3675,-3.1602,-1.7835,-4.3918,0.1171,0.7812,0.6232,2.8998,1.5585,2.3136,-1.0991600000000001,-0.052659999999999985,0.0608,-0.4854000000000001,-0.06359000000000004,-0.3138699999999999,1.248331356010895,0.4135176905526534,0.2879912463947472,2.1578766368817286,0.9775692747319753,1.8656674623576412,1.6632750464069377,0.41685723695289256,0.29433925664104,2.2117967221243457,0.9796353275581685,1.8918851595696815,1.1142702588690525,0.19460000000000005,0.4614432341237208,1.4276245510242158,0.6359000000000006,2.451266884736691,10.9916,2.351235884682489,1.2351822682124667,13.854707194355028,4.801287314033497,11.0622215982954,253.46273616000002,8.846086159999999,5.0811712799999995,259.43680398000004,52.73186716000001,188.7547036999999,-0.2928,-1.3908,-2.0334,-0.8864,-1.1007,-3.4512,2.5634,0.846,0.229,1.1684,0.9024,3.471,1.0572700000000002,-0.31632,-0.91966,0.20330000000000004,-0.10251999999999999,-0.053849999999999995,1.1875867438212673,0.7197689473713075,0.8819262737893684,0.7588039298791224,0.729364764435464,2.416632343262003,1.590025825890888,0.7862096921305409,1.274193261636554,0.7855662250886299,0.7365346631354155,2.4172322406008075,0.2557493071588891,0.8422212804906613,0.4520137849547068,1.3896046111188194,1.0083238723409453,0.5385,11.6705,5.391,9.1966,3.618130855538453,3.3587992703445915,11.76442873398752,250.40038412,50.440641619999994,160.28049736,35.95298120000001,31.407868299999997,311.84370501999996,0.1965,-1.2677,-0.3772,-1.8659,-2.4515,-2.0649,0.5203,-0.7753,-0.0187,1.8694,1.8447,2.9848,0.36158999999999997,-0.99329,-0.17176,-0.23317000000000002,0.07675000000000003,0.03779000000000003,0.1017348657049293,0.1518657331329224,0.12329708187949948,1.254371312690146,1.2166542452562272,1.7349660662099418,0.37562922010940525,1.004832436279801,0.21143241946305205,1.275858784897451,1.219072645497388,1.7353775770707651,0.20278696997321727,0.27110000000000056,0.12780000000000014,1.0111000000000006,0.7675000000000001,0.37790000000000035,3.6159,9.9329,1.7176,7.929929041533852,6.312426969509956,11.11159009253611,13.718585599999999,99.85241006000001,3.7184247,84.62035599999997,81.25356199999999,151.81357739999999,0.1778,-1.4148,-0.6176,-2.1725,-1.1842,-2.3954,0.5823,-0.7684,-0.028,2.7468,1.0098,3.4768,0.35089000000000004,-1.01804,-0.26271000000000005,-0.22224999999999998,-0.14733000000000002,0.002400000000000002,0.13379484631330163,0.19459191247325774,0.180226687535448,1.7584487585653439,0.7349333330989961,1.9385293580443914,0.3755327588906193,1.0364706720404586,0.31858782619554066,1.7724381791757928,0.7495552901554361,1.93853084370613,0.24562910210835218,0.26919999999999966,0.18057981025427938,1.714909914184876,1.1277893094323397,0.0239999999999998,3.5089,10.1804,2.6271,11.75005166313027,3.7278120080011683,12.53050434308289,13.457710380000002,105.5700791,8.668903360000002,164.23238691999998,32.16524213999999,194.90414760000007,-0.3395,-1.1977,-0.4639,-1.9354,-2.6969,-1.2148,-0.1751,-0.6167,-0.2181,1.6226,1.4006,1.1908,-0.22336000000000006,-0.9829100000000001,-0.34865999999999997,-0.14199,-0.031709999999999954,0.08593000000000002,0.04709707421910623,0.17652894068678934,0.07658882686136405,1.174293674044104,1.2594200129027646,0.7689442698271444,0.22827138235004404,0.9986363377125829,0.3569728897269371,1.1828469017586343,1.2598191509101613,0.7737307380478043,0.0041999999999999815,0.1745000000000001,0.05059999999999998,0.3149000000000003,0.31709999999999994,0.5647694136411368,2.2336,9.8291,3.4866,6.632827211964346,9.0033,4.037154921822279,5.0998845,98.18455528000001,12.450952160000002,71.01397865999999,119.9371963,30.50017024,-0.4223,-1.4429,-0.6128,-1.7581,-2.4073,-2.3274,-0.1908,-0.7357,-0.1011,2.227,1.8542,2.0999,-0.30884999999999996,-1.02881,-0.33373,-0.09529000000000001,0.23796,0.14416,0.0816676955717498,0.26480983912989337,0.15002537152095308,1.34662000612645,1.1432691268463435,1.5216817395237414,0.3194650763385569,1.0623437612185616,0.36590070374351563,1.3499872684584844,1.1677710640360979,1.5284951494852708,0.1704877253729143,0.1851110808606297,0.08909999999999973,0.9529000000000003,0.8437150501158951,0.5305999999999996,3.0885,10.2881,3.3373,8.48634881503168,6.154039888310087,9.809701833822633,9.890116719999998,109.54186727999998,12.2669213,94.0898999,84.09936956,117.99474456,-1.7502,-0.6374,-0.1826,-2.3187,-1.0972,-3.4137,0.0151,0.6153,0.2606,2.0184,1.2207,3.6071,-0.83199,0.03932000000000001,0.0047399999999999986,-0.10191000000000003,-0.22880000000000003,0.11513999999999999,0.6468247837706901,0.43484339893805446,0.14359836489319786,1.7757046643234342,0.6479333129265696,2.1282624561834473,1.0538451788569325,0.4366175030847939,0.1436765742910096,1.7786266339510381,0.6871441027906737,2.1313747446190687,0.6654666932978878,0.3932000000000001,0.047400000000000025,1.0191,1.2078000000000002,1.1513999999999998,8.3199,2.518615526377582,0.7768510657884209,8.794568571161372,3.1352251419795945,11.72053855350527,90.89102129999999,10.05103334,1.13815328,168.46532767999994,26.955213319999995,228.48709935999992,-3.6119,-1.1998,-0.3823,-2.5929,-2.1655,-1.9281,0.2288,0.8868,0.4188,2.7797,1.4569,2.7031,-1.44008,-0.12889999999999996,0.07847000000000001,-0.08796999999999996,-0.14499,-0.044479999999999985,1.5161937909119665,0.569408292879547,0.21778593182297154,1.6876248665209932,1.2972538559973525,1.406002024749609,2.091093976367394,0.5838159076284236,0.2314913670096576,1.6899160958461814,1.3053312479979937,1.4067054290077934,0.4599526707136322,0.7965388842130656,0.4218999999999999,0.8797000000000001,1.4499000000000004,0.44479999999999986,14.557,3.199269305003348,0.9316908088955199,8.15425118511761,5.814881512656194,7.296563202416879,428.27834572000006,19.189551119999997,3.0762895,149.26675963999998,101.10174375999998,107.07276534000002,-0.3885,-1.2307,-0.4979,-2.1091,-1.8399,-1.9531,-0.167,-0.5928,-0.1238,1.6637,1.3447,2.0213,-0.25127,-0.9505399999999999,-0.26620000000000005,-0.33202000000000004,0.08160999999999999,0.3299500000000001,0.07001422783977554,0.20559141129920772,0.09380602326076935,1.0843704319096865,1.0125956502474225,1.0965012870489481,0.26084210741366126,0.9725194702421129,0.28224459250798767,1.13406195333412,1.0158790001766942,1.1450685896486725,0.06901501979370299,0.20058919063679692,0.12992678312686773,0.9292000000000007,0.8160999999999999,0.6967000000000001,2.5127,9.5054,2.662,5.463990160242424,4.155990627785059,5.007392237290554,6.731339020000002,92.842559,7.54125628,70.24839604000002,53.96463688,71.24514932000002,-0.1593,-1.9617,-2.4875,-1.0266,-1.8614,-3.5918,2.2896,0.5137,0.4564,1.8106,1.1073,3.7441,0.87447,-0.59882,-1.0853,0.4260300000000001,0.18729,-0.1730899999999999,0.8357507858805758,0.7916804693814292,1.1171618781537438,1.0197836682846024,0.8844771975014392,2.573800582582108,1.2096185998073938,0.9926446282532334,1.557538683949776,1.1051969467022609,0.9040892970276774,2.5796142322060485,0.6139447451708016,1.1699752886883448,0.35551768706211845,0.3242999999999996,1.8662390639470436,1.7308999999999997,8.7447,5.988200000000001,10.853,4.720860703819905,4.7417,12.111601748164901,141.43817334000002,84.75517552000002,237.88112738,70.20067784,53.86460949999999,344.90408896,0.2579,-1.3028,-0.4751,-1.6615,-1.7871,-2.1299,0.6418,-0.6853,0.0436,2.0466,1.6291,3.245,0.4102,-0.92768,-0.20342,-0.33775999999999995,0.010780000000000012,-0.29220999999999997,0.14128990763674526,0.18140065490510227,0.18079502648026577,1.1398577605999793,1.2323550890875568,1.5641008589282215,0.43385121643254615,0.9452493745038925,0.2721516819716534,1.1888471440853952,1.2324022370963144,1.5911625250111945,0.24707307656197403,0.39450447397662824,0.1992228007827165,0.9919844475884555,0.10780000000000012,1.6229,4.102,9.276800000000001,2.0342,6.669052138478348,11.8122,9.10211911347841,17.886921020000006,87.79549262000002,5.94109222,81.63243795999999,145.71060854,132.17614546000004,-0.3873,-0.7895,-2.1004,-1.1086,-1.2564,-3.1663,2.4016,1.0347,0.3834,2.2987,2.0409,3.5966,0.9616999999999999,-0.12924000000000002,-0.88264,0.46154,0.01657000000000002,-0.27589,1.1781118724467552,0.6212846693746756,1.0536075656524113,0.8758592537616988,1.0844533969239987,2.150906311976419,1.520794027473806,0.6345846027126723,1.3744607204281978,0.9900245471704225,1.0845799809142707,2.1685279004430633,0.42725284669049235,0.8945287306326452,0.18970442477424554,0.44159999999999977,0.16570000000000062,1.2561000000000004,11.471,2.7309126533217327,10.378999999999998,4.615399999999999,5.459300000000001,10.004135030634735,227.67598870000003,22.465787859999995,187.27160257999995,59.755894059999996,73.73139324000002,239.72032095999998,-0.7378,-1.3688,-0.5888,-1.6997,-1.717,-2.9663,-0.1621,-0.5836,-0.1066,2.6178,2.7857,2.4529,-0.45472999999999997,-0.95558,-0.29234000000000004,-0.01490000000000007,0.04042999999999999,-0.10956999999999999,0.16680465251305193,0.2912401030078104,0.15454216382592809,1.2548154509727716,1.2515008414300008,1.876650604694438,0.4843585087515238,0.9989764431657036,0.33067500056702204,1.2549039110625164,1.2521537209943514,1.8798465567699933,0.1257478220068149,0.023835111539512014,0.08969760598499592,0.14900000000000013,0.4043000000000001,1.0957,4.5473,9.555799999999998,2.9234,7.328484640384734,9.1571,11.139583000239917,22.550798020000002,95.58220151999996,9.76656056,79.65641902,120.40241649999999,178.22482093999994,-0.5213,-1.1995,-0.3788,-1.4349,-1.4969,-1.4286,-0.2265,-0.6989,-0.0313,1.7073,1.5954,2.0054,-0.35113000000000005,-0.94933,-0.17123,-0.022680000000000033,0.23742000000000002,0.0872099999999999,0.1033293573966276,0.18902152284858992,0.09170655429139185,0.9858742148976207,0.9021981853229367,1.2308150490223948,0.36601807742241366,0.9679651775761358,0.19424161500564188,0.9861350566732734,0.932914691705517,1.2339008343460993,0.12433971407259606,0.2822545958220643,0.15150000000000008,0.2268,2.3742,0.8720999999999998,3.5113,9.4933,1.7123,5.811309999366159,3.8485999999999994,6.618124588428366,13.235891619999999,92.03991554000001,3.3639520199999997,49.94295534000001,53.740764899999995,78.46307377999999,-2.6324,-0.7308,-0.549,-2.5234,-2.4714,-3.7427,0.055,1.4194,0.3155,2.7672,2.1854,3.168,-0.8509400000000001,0.09468,-0.10109000000000001,0.20728,0.03224999999999999,0.11646999999999996,0.79502218610552,0.6826255134405687,0.274763135991712,1.6498568457899612,1.5364651868818895,2.1798355713447743,1.1645424680963765,0.6891602817922693,0.29276948099144484,1.6628266927133448,1.5368036091186148,2.182944886844375,0.17359999999999953,0.5313999999999998,0.09819944334063387,2.0728,0.3225000000000007,1.1646999999999998,8.5094,3.1727123122168717,1.3133431346482112,8.196537879975043,12.1753,10.226450158141235,104.02797065999998,24.336502799999995,4.90472386,143.31737489999998,192.25923482000002,247.81831584,-0.0164,-0.9643,-0.7466,-1.2769,-2.1095,-3.7333,1.0214,0.4658,0.0909,1.3829,2.1439,4.3761,0.56586,-0.28701,-0.35735,0.08376999999999998,0.07068999999999995,0.3865100000000001,0.37592266545128666,0.5614696099523108,0.2634924486584008,0.8641826335329819,1.2748627968922772,2.7666504912800245,0.6793492400820067,0.6305734398149037,0.44399019471154993,0.8682332848952521,1.2768211413506592,2.7935183767070515,0.3938357409271427,0.37650000000000006,0.362913202834882,0.8377000000000001,0.7068999999999994,2.6919000000000004,5.6586,3.340681888571557,3.5735,4.0337704635907725,6.5788492414459006,14.84623396623276,43.675490499999995,24.07075628,18.102415020000002,40.15445153999999,90.0634254,401.2799078599999,0.0468,-1.2615,-0.4121,-1.8603,-0.838,-2.0528,0.5048,-0.8236,-0.1011,1.599,0.9363,2.4884,0.27608,-0.9991000000000001,-0.24073000000000003,0.026670000000000017,0.2911,0.19587999999999994,0.1191311949071275,0.16427159827553878,0.1109692935004995,1.1543386037467516,0.5750571815045874,1.360251875793597,0.3006865610565261,1.0125146754492007,0.2650756816458273,1.1546466563412376,0.6445385729341573,1.3742831367662198,0.05259999999999998,0.3063864741801666,0.045833611050239115,0.26669999999999994,1.2215015277220755,1.9587999999999997,2.7607999999999997,9.991,2.4073,7.6938575815594925,2.911,8.52709425772453,8.3330121,101.27458761999998,6.531588620000001,68.7118407,27.121204779999996,101.56889779999999,-0.0787,-0.8219,-1.8529,-1.6576,-1.5564,-3.1035,2.1428,0.6773,0.4391,1.8974,3.3975,2.7824,0.6574099999999999,-0.19183999999999996,-0.6235999999999999,-0.18465000000000004,0.39175000000000004,0.31094999999999995,0.7685163101066885,0.5390916029767112,0.7510886219348553,0.9627916049176997,1.45001236356798,2.0401783202700687,1.0113383345844256,0.5722083029806541,0.9762228628750711,0.9803383584252939,1.501999972370173,2.0637387143240784,0.8902407183208639,0.08657387216756222,0.2433198381986945,1.8465000000000003,2.7451,1.7689,6.5741,3.0666580400589027,6.8752,5.00030477259955,5.859320126191088,9.7341914576951,88.79520275999998,18.380259359999993,90.72858942,58.38040461999999,124.24138598000002,219.34987277999994,-0.624,-1.4327,-0.4493,-1.8275,-2.8706,-2.4044,-0.075,-0.5156,-0.0822,1.9998,2.9393,1.5511,-0.37119,-1.00119,-0.25483999999999996,-0.12130999999999999,0.08102999999999998,0.04979,0.16501972882052618,0.2763502793557481,0.12998975498092147,1.1991715293901868,1.7426955798704489,1.3061166880872472,0.4062185704765354,1.038629333785639,0.28607824454159386,1.2052918621645132,1.7445783860291286,1.307065356820385,0.13637760152613157,0.24757886776077076,0.045172471816554184,0.5490007236312897,0.8102999999999998,0.49790000000000023,3.7119,10.0119,2.5484,7.1689301067490065,15.6869,7.589150940401564,15.8129252,105.28138446000003,7.481040760000001,90.11749266,275.54539609999995,94.58767320000001,-0.5509,-1.2274,-0.3627,-1.1477,-0.8982,-1.8304,-0.1967,-0.7231,-0.0774,1.7902,1.5695,1.7014,-0.35296999999999995,-1.00673,-0.20207999999999998,0.08393999999999995,0.35092,0.03904999999999999,0.1370861776401983,0.20449896356705577,0.09821112767909755,1.167346168195193,0.7993930857844593,1.2862062775853647,0.3786560985907925,1.0272901824703669,0.22468144560688585,1.1703602009637888,0.8730258598689961,1.2867989318459974,0.061948545215060706,0.1595000000000013,0.12188421045811361,0.2705999999999996,0.5883053164193615,0.3905000000000003,3.5297,10.0673,2.0208,6.819382382453491,4.02154071528223,7.00241983370548,14.171807940000003,103.45424071999999,4.575872639999999,68.87605835999999,52.04509559999999,84.42237532000001,0.0557,-1.4577,-0.4127,-1.9852,-1.8822,-2.2189,0.3733,-0.6464,-0.0452,1.8345,1.304,2.562,0.20651000000000003,-1.04957,-0.22353,-0.25544000000000006,-0.12835000000000002,-0.18378000000000005,0.09563437091339076,0.2974892873701505,0.11392137683507869,1.215836117410566,0.9554668306644664,1.4523113686809725,0.2275792455387793,1.0909156983928685,0.25088591231872703,1.242379594970877,0.9640490583990008,1.4638932338118105,0.24045155699895857,0.23607825163014173,0.04489999999999994,0.6808000000000001,0.8482060487989114,1.1504000000000003,2.0651,10.4957,2.2352999999999996,7.73600647946068,4.192641284865113,8.843781487866373,4.764537460000001,114.76742241999997,5.646478099999998,80.6695769,48.84571368000001,109.49963450000003,0.2407,-1.3297,-0.4389,-2.3601,-1.7542,-2.092,0.4325,-0.8114,-0.0301,1.946,0.9626,2.7572,0.32588999999999996,-1.01176,-0.20146999999999998,-0.23360000000000003,-0.07036000000000002,-0.009880000000000066,0.06965514266728624,0.14083584912940317,0.1367916156056357,1.4757532951005057,0.777092900752542,1.6047589593456084,0.3332508529621492,1.0215150679260683,0.24352023940526998,1.4941274202691013,0.7802716872987254,1.6047893730954228,0.15799652870548858,0.10839999999999961,0.01429999999999998,0.9394241089874726,0.7036000000000001,0.0988,3.2588999999999997,10.1176,2.0147,9.694541432860714,3.806142392802858,10.411001737353924,10.902924279999997,103.36344185999998,4.994715640000001,115.25918589999995,31.27776836,131.4228502,-3.8717,-1.0523,-0.6069,-2.6359,-4.3465,-3.7196,0.3205,1.154,0.451,3.1671,1.6782,3.027,-1.3516400000000002,0.17239999999999997,0.01825,0.33582999999999996,-0.34724999999999995,0.51717,1.701773128357596,0.6940493440671204,0.26790611881776794,1.9176286783681558,1.7175779325841374,1.998945859221805,2.173237784965097,0.7151407218163429,0.26852700236661486,1.9468131746523598,1.7523288838000703,2.0647637533141654,1.8276902561312527,0.1968000000000003,0.11550000000000002,2.4326999999999996,1.5624988458194768,0.8838999999999992,15.7798,3.9777607441678837,1.2448317757649021,9.240610147152251,8.4081,10.40862625908307,451.99570199999994,27.076765719999997,3.6286607999999996,198.10218093999998,194.91002677999998,226.92634789999997,-0.5665,-1.8014,-2.1281,-0.767,-3.6651,-3.2458,3.1033,1.2111,0.3519,1.736,0.8043,5.688,0.9402999999999999,-0.38377,-0.7656099999999999,0.41050000000000003,-0.81709,-0.06255000000000008,1.2280142230446682,0.9395403866252904,0.86300072821522,0.8616190712838243,1.4044917824252299,2.7009560267616357,1.5466683619961974,1.0148968179081062,1.153658930967034,0.9544095944614136,1.6248794524517811,2.701680210720729,1.8857256888176837,1.041597595393496,1.2722611618320958,0.4767373373701679,2.706705747088018,0.6255000000000006,9.827600000000002,5.0601,7.6561,5.675992801582248,9.731300000000001,13.170217785872236,212.10821648000007,71.6670542,120.11626145999999,54.99742731999999,212.74256500000004,370.60135538,-0.5039,-1.4473,-0.5014,-1.6493,-2.1616,-2.3678,-0.1522,-0.7304,-0.1231,1.9215,1.5791,2.6952,-0.34135,-1.00552,-0.26938,-0.046499999999999986,0.10324,-0.06162000000000001,0.12190822162594286,0.22660163635772804,0.11007926961966998,1.1277626567678147,1.0541098455094706,1.6493895196708386,0.36246577355662146,1.030737004283828,0.29100348795160513,1.1287208955273222,1.0591534657451676,1.650540157645369,0.21927960915135136,0.43601871605104997,0.22179999999999978,0.46500000000000014,0.9803762209103366,0.543513514107865,3.4135,10.0552,2.6937999999999995,7.713396813466525,6.489819025633424,11.092214637405958,12.483978420000001,104.09564053999999,7.887028339999997,64.87095438000001,58.481048959999995,140.05817759999994,-3.4651,-1.0454,-1.2296,-2.7939,-1.929,-6.1274,0.0496,1.0706,0.2583,1.0265,3.7093,4.6788,-1.21193,-0.09242000000000002,-0.35388000000000003,0.06615,0.2124,0.00419000000000005,1.3725287013756762,0.6688891504576824,0.4869327813980077,1.1273639334749004,1.5069537550966852,2.924849909807339,1.8310132061238662,0.6752437722778345,0.6019423460764328,1.1293029978708105,1.5218486718461859,2.9248529110025343,1.232451679307283,0.9242000000000004,0.9307525572920609,0.6614999999999998,2.123999999999999,0.04190000000000005,12.119299999999999,3.7752000000000003,3.5388,5.448555954291252,7.254901291628378,17.917597206744436,247.09206353999994,30.35084794,25.1791538,69.31941201999999,122.49765301999999,432.12790099999995,-0.5038,-1.2888,-0.3488,-0.9338,-1.5173,-2.6724,-0.2289,-0.5198,-0.1062,1.9321,1.2174,1.425,-0.37438000000000005,-0.9524900000000001,-0.21948000000000004,0.22152,-0.29812000000000005,-0.26713000000000003,0.0961649811521845,0.25951595885417145,0.07779193788561897,0.7734692880780722,0.7927239756687066,1.18114083161154,0.3865334241692431,0.9872110883696555,0.23285844627154928,0.8045656281497489,0.8469278812272035,1.2109715525147566,0.13501209015401877,0.20909314209320426,0.04999999999999982,2.0387960978084188,0.7435999999999996,2.2339564420261704,3.7438,9.5249,2.1948,3.46614971568759,4.365407232555444,4.7561432668869905,14.780288119999998,94.15097006000002,5.12097632,38.201928339999995,40.58458900000001,86.2048865,-0.66,-1.6733,-0.5965,-2.178,-3.2379,-2.7478,-0.2142,-0.6587,-0.1036,2.3039,1.7081,2.0588,-0.40472,-0.9973899999999999,-0.32452000000000003,-0.19307000000000002,0.017290000000000007,0.16939,0.1300784824634728,0.30772482009093777,0.164803281520727,1.3875197325083346,1.360155956829951,1.6721786121404614,0.4251102092399099,1.043782245968957,0.36396888878034617,1.4008879444837834,1.36026584570811,1.6807362324291102,0.2089634593714635,0.16433864572543114,0.13345157566055618,1.9307000000000003,0.17290000000000028,1.6939000000000006,4.0472,9.9739,3.2452,8.554238027922267,8.9909,10.201864773841606,17.316756899999998,104.41255306000001,11.90347474,104.19692373999997,132.94924715999997,145.32129316,-0.4553,-1.2473,-0.3637,-1.4987,-2.4078,-2.9087,-0.1637,-0.5575,-0.104,2.0711,2.0726,1.7145,-0.32386000000000004,-0.9629800000000002,-0.23894,0.02987000000000003,-0.12574999999999997,-0.11061999999999998,0.09810191843180233,0.23047050483738696,0.09882745772304376,1.164844703855411,1.3038409038299112,1.5762225920218247,0.33839220735708436,0.9901753046809438,0.258571440805051,1.1652276176781942,1.3098908981285426,1.5800995044616652,0.09308904186808256,0.23984370735389285,0.030821302810291933,0.2987000000000002,1.2575000000000003,0.8175865253991198,3.2386,9.629800000000001,2.3894,6.882349581744835,10.0531,8.808432335182339,11.206814259999998,95.43699274000004,6.1985792,71.55931334000002,137.11377118000001,135.05290591999997,0.2645,-1.4807,-0.4085,-1.6151,-1.5971,-2.3602,0.7151,-0.5875,0.0666,1.1146,1.4152,2.3563,0.39968,-0.9926999999999999,-0.13384000000000001,0.021880000000000017,-0.19327,0.15090000000000006,0.12764399555012368,0.2702783528142792,0.13389682744561202,0.7901234457981867,0.8672878300195385,1.6133877649219979,0.41956774423208465,1.0288360792662745,0.18931853052461609,0.7904263368587866,0.8885614627024965,1.6204292301732897,0.17352829066104752,0.31030929294235826,0.04442557106750346,0.21879999999999988,0.8459,0.5277694936259729,3.9968000000000004,9.927,1.3384,4.76449894861967,4.489308680916166,10.899573299519473,16.956448700000003,102.29846938,2.7241286,31.560618420000008,41.702511699999995,133.75751318,-0.0819,-1.0423,-1.3855,-1.2115,-1.3568,-3.5776,1.2528,0.5546,0.1448,1.3448,2.6507,4.3039,0.73495,-0.23148999999999997,-0.69145,-0.06152999999999999,0.60755,-0.10740000000000002,0.47491124697147363,0.5257984204046261,0.47584197849706367,0.6995437185051411,1.2808309906072697,2.4654746488252517,0.8750383963004138,0.5745011740632041,0.8393620738394129,0.7022445122605089,1.4176194937288356,2.467812797600337,0.9005624236821006,0.575477518088646,0.19070137337836174,0.6153000000000002,1.1608501692581121,1.074,7.349500000000001,2.4748124123202815,6.9145,3.3998999999999997,7.756957582947301,14.586292540918288,71.07728600000002,19.780926079999997,66.12351980000001,30.6263248,134.41325501999998,310.41707598,-3.1479,-0.8878,-0.7012,-3.863,-1.601,-3.2894,0.3868,0.7937,0.3829,2.2441,1.1618,3.6703,-0.9504400000000001,-0.005870000000000019,-0.04136000000000001,-0.35011000000000003,-0.06207,0.08550000000000002,1.2332728799418238,0.49143374538181644,0.3212704132035815,1.7851942406640235,0.8433551470762479,2.2476194055044108,1.55701579632321,0.4914688016547948,0.3239217930303548,1.8192018824198704,0.8456362036951823,2.2492450382294944,0.9510771874082876,0.058700000000000196,0.41359999999999997,2.4010109038739285,0.6207,0.7755233596449358,10.5048,2.59169552808269,1.848,9.064397069915401,4.97403936750807,10.283075172140087,221.5571307,13.659621799999996,7.039350880000001,198.72892018,39.3097475,263.46728612,-0.7867,-2.0431,-0.4135,-2.104,-2.5308,-2.1917,-0.1649,-0.5044,-0.1228,1.7697,2.1501,1.1247,-0.41333000000000003,-1.0046799999999998,-0.27837,0.042710000000000026,-0.27816,-0.27841000000000005,0.18454407088822986,0.4706990839166781,0.08603005346970324,1.1674339985198305,1.6497394704619273,1.120675775101791,0.45265682696718496,1.1094771426216947,0.29136064765166897,1.1682149994756958,1.6730251958652624,1.154740802518037,0.12846530255670313,0.5609369319399984,0.050899999999999945,0.42710000000000026,0.8157732596023387,1.6125119212959282,4.1333,10.046800000000001,2.7837,6.785900000000001,15.736,5.5313613626013,18.997660000000003,112.39122284000003,8.1203396,91.35174086,267.63016258000005,78.10869405999999,-2.6984,-0.4736,-0.2092,-2.3889,-2.4407,-1.6779,0.4043,0.4727,0.5299,2.3405,1.4543,2.1478,-1.05815,0.01788,0.07484000000000002,-0.07167000000000004,-0.65488,0.21479000000000004,1.1126988867164378,0.34552759310943604,0.2194986842785168,1.8658635030730408,1.1726029232438404,1.3264141747207017,1.5355065076384407,0.3459899015867371,0.23190665794668336,1.867239460005063,1.3430805746491907,1.3436924153242809,0.8899919424566772,0.17880000000000007,0.3733765826041034,0.638818053877926,1.833201343680469,1.5970195780077736,10.5815,1.3810978323315224,1.0478133409929848,10.90463199856096,7.168199999999999,8.130041870734427,228.50882332,6.04441844,3.0802894600000004,183.33598893999996,137.32820784,96.13571396000003,-2.6092,-0.5015,-0.2213,-2.2348,-1.1428,-1.5215,0.3551,0.631,0.3695,2.0658,1.0017,2.6748,-1.13582,0.10617000000000003,0.14784,0.32353999999999994,0.09379999999999997,0.74619,1.0443688216334304,0.34951739885161653,0.22308120135950496,1.7130676683657304,0.7359826968074725,1.569560485263311,1.5429819532321174,0.3652868475595583,0.2676226597282076,1.74335279504752,0.7419359608483741,1.7379066813267048,1.2774965743791766,0.4888306435698687,0.3928619505193454,2.30845429245114,0.6001999999999998,3.1235,11.3582,1.7953561070101174,1.4784000000000002,10.567633877063676,4.2112577044752175,8.458321213829548,187.25955702000002,7.358981140000001,4.8410853,160.35845579999992,28.143490519999993,183.73408357999995,0.1259,-1.028,-1.1562,-1.0269,-0.7493,-3.3871,0.9626,0.4929,0.1543,2.6695,2.3083,3.7191,0.43804999999999994,-0.39131,-0.59276,0.4023300000000001,0.08203,0.13801000000000002,0.23904020268565704,0.5486393340802316,0.45717432605079644,1.300644025127552,0.9714247351699462,2.81516352684884,0.4990270744158077,0.6738906699161222,0.7485805113680156,1.361449341327102,0.9748820118352785,2.8185443837200794,0.13983721177006067,0.030100000000000016,0.3968279208394391,1.157543666066856,0.8203000000000003,1.3801,4.3805,3.9131,5.9276,7.9309,4.902611585469902,14.866362609987313,23.324351579999995,30.36306056,53.988580299999995,132.22027430000003,51.293372699999985,418.89554028,-3.9846,-0.6451,-0.6042,-2.9502,-1.825,-4.6219,0.3037,1.3637,0.6071,3.2113,3.3306,2.9407,-1.07142,0.2333,-0.03674999999999999,0.21111,0.5529200000000001,0.10823999999999996,1.4294140602358716,0.7423217550361837,0.3515056279777039,1.95978982671612,1.4068095974935626,2.1727579392099803,1.7863833216865859,0.7781198352439038,0.35342151745472433,1.9711274938471128,1.5115666607860865,2.175452357556929,0.7278792908461236,0.9615986203172712,0.0002999999999998837,2.108798279299535,1.2421687284915304,1.0823999999999998,11.671000000000001,3.7242343432447864,1.731029264327161,10.268669796386645,6.958599999999999,9.877127848353012,285.06142992,35.9668904,6.312866619999999,205.15209586000003,153.73877179999997,243.54725986000005,-2.689,-0.7819,-0.797,-3.7036,-1.421,-2.9789,0.2231,0.7815,0.136,2.585,2.5161,2.9508,-0.8328900000000001,0.03957999999999998,-0.24286,-0.42497000000000007,-0.17165000000000002,-0.10519999999999996,0.9908443272785085,0.5227564381239125,0.29487881985656417,1.800607825180153,1.1163477578693835,1.8107673207786803,1.2944026548952994,0.5242526776278783,0.3820137405905709,1.850077847281027,1.12946714826063,1.8138206443857674,0.8138488551174845,0.3957999999999998,0.5817440912018513,1.819238115991513,1.7165000000000004,0.1237999999999998,8.444700000000001,2.9936934747106054,2.4286,9.589587626647768,6.7453,10.117691024806367,154.11567830000004,15.82437432,12.36139838,186.15759934,88.00752412,165.05828172,0.0322,-1.2916,-1.1275,-1.01,-2.917,-3.9356,1.9672,0.6344,0.1661,1.6138,3.2849,3.9238,0.66412,-0.5630400000000001,-0.49699,0.39812000000000003,0.02297,-0.0739,0.5572346252701819,0.6887591613909757,0.41670749321316497,0.8601815201456028,1.918274466310804,2.7959348078952053,0.8669289486457353,0.8896084666863282,0.6485708866423161,0.9478458640517454,1.9184119862532134,2.7969112713849182,0.8008571515872909,0.12037946777160799,0.28262201306857637,0.16100000000000048,0.2296999999999998,0.7389999999999994,6.6412,5.6304,4.9699,5.315893009927638,10.469659471161236,13.562648851579219,69.07926332000001,57.86910519999999,40.0481586,52.85852632,189.43820654,416.74735021999993,0.049,-1.2231,-0.4234,-2.9353,-2.114,-2.2869,0.4499,-0.7924,-0.0736,1.5832,1.4426,2.4872,0.30001,-0.9828200000000002,-0.19823,-0.16743000000000002,0.19911000000000006,0.13219,0.10192314212189497,0.13714845824871674,0.09876980358388894,1.5405958685197103,1.0929196717508565,1.7017979782864945,0.316850638314017,0.992343112033333,0.22147371627351178,1.5496672013693777,1.1109087275739624,1.7069242967981912,0.14485281810897166,0.07460000000000022,0.009299999999999975,1.361325390948854,0.5957829977195703,0.6453151220346468,3.0000999999999998,9.828199999999999,1.9823,10.230314589277487,5.566730383836252,11.612719916240524,9.63382856,97.53678279999998,4.41733024,122.7186842,66.04194869999999,147.28178275999994,-0.6302,-1.5237,-0.4091,-1.2175,-0.6496,-2.5348,-0.1221,-0.6185,-0.1179,0.7637,2.0867,1.5707,-0.44708999999999993,-0.9727500000000001,-0.25385,-0.15841,0.48388,0.23544000000000004,0.15933682217240308,0.3038184334433972,0.09894748354556572,0.674640997049542,0.9520568426307329,1.3018616702246057,0.47463427078119846,1.0190918520918515,0.27245261422860306,0.6929893238715874,1.0679663318663186,1.3229798947829858,0.3189726088384072,0.28847451232555393,0.08971447690680584,0.5962327289830073,0.6655516555383127,1.667735050470052,4.4709,9.7275,2.5385,3.3509964022008387,6.0124,6.6380556883347746,21.639160599999997,99.67693132,6.94582128,30.54797156,86.80907390000002,92.77719466,-0.2907,-1.251,-0.3399,-1.3556,-0.5377,-1.3309,-0.1339,-0.7275,-0.1178,1.3842,1.1023,0.9088,-0.23773999999999998,-0.96319,-0.2730100000000001,-0.14870999999999995,0.11699000000000001,0.06448,0.050185280710582855,0.18223372602238036,0.06605893505045324,0.8254041415573343,0.5640714395358092,0.6633313090756383,0.2429791554845806,0.9802775663045645,0.2808883105435326,0.8386934249175917,0.5760757320005765,0.6664578726371233,0.11124136861541335,0.04174383490308014,0.11968847731944786,0.838480195590701,0.44131924844880355,0.6447999999999998,2.3773999999999997,9.631899999999998,2.7300999999999997,4.840011537341846,3.3499977747677057,3.6540774473954576,5.793729759999999,94.44138395999995,7.679916999999998,36.68748146,17.560774099999993,23.5045909,0.0815,-1.2115,-0.3895,-1.7785,-1.8645,-1.6939,0.5556,-0.7202,-0.0376,1.3214,2.3763,1.881,0.35301,-0.95837,-0.18869,-0.07631,0.021660000000000023,0.021660000000000023,0.14443018001788963,0.16659776739200322,0.11353521436100784,1.0468934018800578,1.2657677442564255,1.331369238190518,0.3814133414027359,0.9727424494695398,0.22021389828982182,1.0496709060462712,1.2659530552117642,1.331545419428117,0.059700000000000086,0.3271000000000006,0.0532999999999999,0.7631000000000003,0.21659999999999968,0.21660000000000013,3.5301,9.5837,1.8869,6.50436300369476,7.475799164273938,8.052443676130432,13.5063919,93.2885437,4.2063242999999995,60.98582856000001,87.9794915,95.03800285999998,-0.5155,-1.828,-2.9163,-2.4891,-1.6701,-3.1342,1.7128,1.0335,0.7456,1.5917,3.3473,3.1265,0.53955,-0.64723,-1.09644,-0.29052,0.19645999999999994,-0.50683,0.8081753810281528,0.9279260434431184,1.4412983939490114,1.4126155172586772,1.2150369238833854,1.8340681852373975,0.9717312637761534,1.1313502618552753,1.8109449837032598,1.4421804568083705,1.2308173130079054,1.902809175140797,0.6045168099643953,0.7131423384731819,0.26462189228717453,0.8448299833741987,0.7549999999999999,3.2718999999999996,7.3383,7.8401000000000005,13.9138,6.723599999999999,5.640723453278435,9.107915539894623,88.69411602000001,115.6765884,320.8820356,130.81771549999996,77.96040198,199.2306351,-0.5881,-1.2705,-0.3715,-1.2958,-1.2658,-3.0171,-0.134,-0.5531,-0.0549,1.6584,1.4186,1.5763,-0.32185,-0.98758,-0.21921,0.20602,0.055730000000000016,-0.31334999999999996,0.1544379891736486,0.23974576033790465,0.11394785166908589,1.025097577599323,0.9419344308920871,1.3818896882530096,0.35698531482401347,1.016263886005992,0.24705695092427576,1.045595181702747,0.9435816366377634,1.4169711828403568,0.0810204630278546,0.25812696159504506,0.08289999999999975,0.7587868085525613,0.5573,2.294813848294781,3.2184999999999997,9.8758,2.1921,5.749123364406666,4.191152278388556,6.262461277459086,12.29537488,100.44380249999999,5.457944259999999,60.526882799999996,47.22256634,119.27226162,-0.7239,-1.4676,-0.399,-0.931,-1.8634,-3.6356,-0.2436,-0.4987,-0.1133,1.0636,1.6271,2.2817,-0.4460800000000001,-1.0298500000000002,-0.25355,-0.04614000000000003,0.05339000000000001,-0.21288999999999997,0.14290097830315926,0.29319158668010925,0.09025055401492005,0.7446405497956716,0.9686991777120492,2.029801279165032,0.4684101365256734,1.070771837974832,0.2691333591363211,0.7460686617195498,0.9701693609880698,2.040934929143994,0.14974888128706917,0.43384665356283814,0.047709067320013676,0.46140000000000003,0.42607131230790385,1.2436840117887356,4.4608,10.2985,2.5355,3.981412401999187,5.7869,10.706650514928612,21.049413440000002,110.4581927,6.837344499999999,29.689184999999995,63.94805986,215.85817750000007,-3.8958,-0.7679,-0.2938,-2.6912,-1.7324,-3.504,-0.4938,0.8522,0.3954,3.3672,2.5054,2.5394,-1.3796599999999999,0.21356999999999995,0.06564,0.06668000000000003,-0.10734000000000003,-0.22063999999999995,1.032708648361192,0.6146400475888306,0.16905357848918784,2.2257507024821983,1.374912267892028,1.942307087563653,1.7233539589997176,0.650687738473686,0.1813497229112854,2.226749292578758,1.3790959430003409,1.95479892367476,1.718752427798326,0.5949200700227674,0.2176,0.6668000000000003,1.0734,2.2064,13.796600000000002,3.2041563070930392,0.9235877289958928,12.128246976308338,6.401,9.010831932078432,263.76706120000006,23.683783339999998,1.8834914600000001,271.6425573999999,116.15777527999998,197.47675535999997]

test_output = [1,1,0,0,1,0,1,1,1,1,1,0,0,1,1,0,0,1,0,1,0,1,0,1,0,1,0,0,1,1,0,1,0,1,0,0,0,1,1,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,1,1,0,1,0,1,0,0,1,1,0,1,0,0,0,0,1,1,0,1,0,0,0,0,1,1,0,1,1,1,1,1,1,0,0,0,0,1,0,0,1]

pred_outputs = []

for i in range(TEST_CASES_NUMBER):
    input = np.array(test_case[INPUT_NODES*i : INPUT_NODES*(i+1)], dtype=np.float32)
    pred_output = get_fpga_output(input)
    pred_outputs.append(pred_output)
    
print(np.count_nonzero(test_output==pred_outputs))

In [ ]:
del input_buffer, output_buffer